In [8]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.tsa.arima_model import ARIMA

In [2]:
# configure df options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

pd.options.display.float_format = '{:,.5f}'.format

In [3]:
df = pd.read_csv('../../dataset/final/training.csv', parse_dates=['date'])

In [4]:
# set index to time-series based 'date'
df.set_index('date',inplace=True)

In [5]:
df.index

DatetimeIndex(['2002-01-01', '2002-01-01', '2002-01-01', '2002-01-01',
               '2002-01-01', '2002-01-02', '2002-01-02', '2002-01-02',
               '2002-01-02', '2002-01-02',
               ...
               '2018-05-31', '2018-05-31', '2018-05-31', '2018-05-31',
               '2018-05-31', '2018-05-31', '2018-05-31', '2018-05-31',
               '2018-05-31', '2018-05-31'],
              dtype='datetime64[ns]', name='date', length=13500, freq=None)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13500 entries, 2002-01-01 to 2018-05-31
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   code    13500 non-null  object 
 1   min     13500 non-null  float64
 2   max     13500 non-null  float64
 3   x       13500 non-null  float64
 4   y       13500 non-null  float64
 5   cm      13500 non-null  float64
dtypes: float64(5), object(1)
memory usage: 738.3+ KB


In [7]:
# sort by date index
df.sort_index(inplace=True)

df.head()

,code,min,max,x,y,cm
date,,,,,,
2002-01-01,ANTHUAN,3.50000,6.00000,106.57215,9.99752,0.00000
2002-01-01,LOCTHUAN,0.00000,0.00000,106.59743,10.24030,274.32000
2002-01-01,SONDOC,0.10000,0.30000,106.52635,10.02522,30.48000
2002-01-01,BENTRAI,0.20000,1.00000,106.53715,9.87584,0.00000
2002-01-01,BINHDAI,0.50000,2.00000,106.69681,10.20635,152.40000


### ARIMA Self Help

In [9]:
help(ARIMA)

Help on class ARIMA in module statsmodels.tsa.arima_model:

class ARIMA(ARMA)
 |  ARIMA(endog, order, exog=None, dates=None, freq=None, missing='none')
 |  
 |  Autoregressive Integrated Moving Average ARIMA(p,d,q) Model
 |  
 |  Parameters
 |  ----------
 |  endog : array_like
 |      The endogenous variable.
 |  order : iterable
 |      The (p,d,q) order of the model for the number of AR parameters,
 |      differences, and MA parameters to use.
 |  exog : array_like, optional
 |      An optional array of exogenous variables. This should *not* include a
 |      constant or trend. You can specify this in the `fit` method.
 |  dates : array_like, optional
 |      An array-like object of datetime objects. If a pandas object is given
 |      for endog or exog, it is assumed to have a DateIndex.
 |  freq : str, optional
 |      The frequency of the time-series. A Pandas offset or 'B', 'D', 'W',
 |      'M', 'A', or 'Q'. This is optional if dates are given.
 |  
 |  
 |  Notes
 |  -----
 |